## Imports

In [ ]:
import numpy as np
import math
import igl
import meshplot as mp
import wildmeshing as wm
import polyfempy as pf

## Read the Mesh

In [ ]:
V, F = igl.read_triangle_mesh("mesh.obj")
mp.plot(V, F)

## Tetrahedralization with Wildmeshing

In [ ]:
wm.tetrahedralize("mesh.obj", "out.mesh", mute_log=True)

## Loading the Mesh in Polyfem

In [ ]:
solver = pf.Solver()
solver.load_mesh_from_path("out.mesh")

## Advanced Sidesets

In [ ]:
minn = np.min(V, axis=0)
maxx = np.max(V, axis=0)

center = (minn+maxx)/2

Function to select sidesets:

- Sideset 2: $z$-component close to the bottom boundary `minn[2]` and larger than $y$ of center
- Sideset 3: $z$-component close to the bottom boundary `minn[2]` and smaller than $y$ of center

In [ ]:
def sideset(p):
    if abs(p[2] - minn[2]) < 0.5:
        if p[1] > center[1]:
            return 2
        else:
            return 3
    return 1

## Loading a Visualizing Sidesets

In [ ]:
solver.set_boundary_side_set_from_bary(sideset)
ps, ts, s = solver.get_boundary_sidesets()

col = np.zeros_like(s); col[s==2] = 2; col[s==3] = 3
mp.plot(ps, ts, col, shading={"wireframe": False})

## Setup Problem

In [ ]:
settings = pf.Settings()
problem = pf.Problem()

settings.set_pde(pf.PDEs.LinearElasticity)

settings.set_material_params("E", 200)
settings.set_material_params("nu", 0.35)


problem.set_displacement(2, [0, 0, 0])
problem.set_force(3, [0, 0.5, 0])

settings.set_problem(problem)

## Solve!

In [ ]:
solver.settings(settings)
solver.solve()

## Visualization of Results

In [ ]:
p, t, d = solver.get_sampled_solution()
m = np.linalg.norm(d, axis=1)

mp.plot(p+d, t, m, shading={"wireframe": False})

## Isolines

In [ ]:
p_uni, indices, inverse = np.unique(p, return_index=True, return_inverse=True, axis=0)

t_uni = np.array([inverse[t[:, 0]], inverse[t[:, 1]], inverse[t[:, 2]], inverse[t[:, 3]]]).transpose()
d_uni = d[indices, :]
m_uni = m[indices]

In [ ]:
adj, _ = igl.tet_tet_adjacency(t_uni)

igl_faces = [[0,1,2], [0,1,3], [1,2,3], [2,0,3]]
surf = []
for t in range(adj.shape[0]):
    for f in range(adj.shape[1]):
        face = igl_faces[f]
        if adj[t, f] == -1:
            surf += [[t_uni[t, face[0]], t_uni[t, face[1]], t_uni[t, face[2]]]]
surf = np.array(surf)

In [ ]:
iso_p, iso_l = igl.isolines(p_uni+d_uni, surf, m_uni, 20)

## Plot!

In [ ]:
p = mp.plot(p_uni+d_uni, surf, m_uni, shading={"wireframe": False}, return_plot=True)
p.add_edges(iso_p, iso_l, shading={"line_color": "white"})
p